In [23]:
%pip install requests
%pip install pandas

import requests
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [50]:
latest = 'https://prices.runescape.wiki/api/v1/osrs/latest'
mapping = 'https://prices.runescape.wiki/api/v1/osrs/mapping'
five_minute = 'https://prices.runescape.wiki/api/v1/osrs/5m'
one_hour = 'https://prices.runescape.wiki/api/v1/osrs/1h'

# get latest high and low prices and unix timestamp from when the last transaction was placed.
# if there has not been an instant buy/sell high/highTime, low/lowTime will be nan
latest_data = requests.get(latest).json()    
latest_df = pd.DataFrame(latest_data['data']).T
latest_df = latest_df.reset_index()
latest_df['index'] = latest_df['index'].astype(int)

# gives a list of objects containing name, id, examine text, buy limit, member status, low/high alch.

mapping_data = requests.get(mapping).json()
mapping_df = pd.DataFrame(mapping_data)
mapping_df = mapping_df.rename(columns={'id':'index'})

#Gives 5-minute average of item high and low prices as well as the number traded for the items that we have 
# data on. Comes with a Unix timestamp indicating the 5 minute block the data is from.
five_minute_data = requests.get(five_minute).json()
five_minute_df = pd.DataFrame(five_minute_data['data']).T
five_minute_df = five_minute_df.reset_index()
five_minute_df['index'] = five_minute_df['index'].astype(int)    
five_minute_df = five_minute_df.rename(columns={'avgHighPrice': '5m_avgHighPrice',
                                                'highPriceVolume':'5m_highPriceVolume',
                                                'avgLowPrice':'5m_avgLowPrice',
                                                'lowPriceVolume':'5m_lowPriceVolume'})

# Gives hourly average of item high and low prices, and the number traded.
one_hour_data = requests.get(one_hour).json()
one_hour_df = pd.DataFrame(one_hour_data['data']).T
one_hour_df = one_hour_df.reset_index()
one_hour_df['index'] = one_hour_df['index'].astype(int)
one_hour_df = one_hour_df.rename(columns={'avgHighPrice': '1h_avgHighPrice',
                                          'highPriceVolume':'1h_highPriceVolume',
                                          'avgLowPrice':'1h_avgLowPrice',
                                          'lowPriceVolume':'1h_lowPriceVolume'})

all_df = pd.merge(latest_df, mapping_df, on='index', how='inner')
all_df = pd.merge(all_df, five_minute_df, on='index', how='inner')  
all_df = pd.merge(all_df, one_hour_df, on='index', how='inner')
all_df['name'] = all_df['name'].astype(str)
all_df = all_df.drop(columns=['examine', 'members','lowalch','highalch','icon'])

In [51]:
items = all_df['name'].tolist()
print(items)

['Cannonball', 'Cannon base', 'Cannon stand', 'Cannon barrels', 'Cannon furnace', 'Bucket of wax', 'Candle', 'Iron arrowtips', 'Steel arrowtips', 'Mithril arrowtips', 'Adamant arrowtips', 'Rune arrowtips', 'Opal bolt tips', 'Arrow shaft', 'Headless arrow', 'Oak shortbow (u)', 'Oak longbow (u)', 'Willow longbow (u)', 'Yew longbow (u)', 'Yew shortbow (u)', 'Magic longbow (u)', 'Guam potion (unf)', 'Marrentill potion (unf)', 'Tarromin potion (unf)', 'Harralander potion (unf)', 'Ranarr potion (unf)', 'Irit potion (unf)', 'Avantoe potion (unf)', 'Kwuarm potion (unf)', 'Cadantine potion (unf)', 'Torstol potion (unf)', 'Strength potion(4)', 'Strength potion(3)', 'Attack potion(1)', 'Restore potion(3)', 'Restore potion(2)', 'Defence potion(3)', 'Prayer potion(3)', 'Prayer potion(2)', 'Prayer potion(1)', 'Super attack(2)', 'Super strength(3)', 'Super strength(2)', 'Super defence(3)', 'Super defence(2)', 'Ranging potion(3)', 'Ranging potion(2)', 'Ranging potion(1)', 'Antipoison(3)', 'Grimy guam 

# EDA

In [43]:
m = set(mapping_df['index'])
l = set(latest_df['index']) 

print(f'Number of items in mapping: {len(m)}')
print(f'Number of items in latest: {len(l)}')
print(f'Number of items in both: {len(m.intersection(l))}')

key_count_m = mapping_df['index'].value_counts()
key_count_l = latest_df['index'].value_counts()
print('number of keys in mapping: ', len(key_count_m))
print('number of keys in latest: ', len(key_count_l))

Number of items in mapping: 4140
Number of items in latest: 4050
Number of items in both: 4034
number of keys in mapping:  4140
number of keys in latest:  4050
